In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from joblib import Parallel, delayed

In [2]:
pubchem = pd.read_csv(
    '../DrugCell/data/SMILES_from_PubchemID.txt',
#     index_col=0,
    header=None,
    sep='\t'
)

In [3]:
p = pd.read_csv('../DrugCell/data/drugcell_test.txt', header=None, sep='\t')

In [4]:
p[p[1] == pubchem[pubchem[0] == 24360][1][1058]]

,0,1,2
331,NCIH929_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,CC[C@@]1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=...,0.452772
505,HCC44_LUNG,CC[C@@]1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=...,0.379520
773,OACP4C_OESOPHAGUS,CC[C@@]1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=...,0.593324


In [5]:
hidden = !ls ../DrugCell/code/Hidden_sample/ | grep 'GO:'

In [6]:
hid_773 = []
hid_505 = []
for i in hidden:
    hid_773.append(pd.read_table(
        '../DrugCell/code/Hidden_sample/' + i,
        sep=' ',
        header=None
    )[0][773])
    hid_505.append(pd.read_table(
        '../DrugCell/code/Hidden_sample/' + i,
        sep=' ',
        header=None
    )[0][505])

In [7]:
f = pd.DataFrame([
        hid_773, 
        hid_505
    ], 
    columns=[i.split('.')[0] for i in hidden], 
    index=['HCC44_LUNG', 'OACP4C_OESOPHAGUS']
).T

In [8]:
f.to_csv('go_weight.csv')

In [9]:
go = pd.read_table(
    '../DrugCell/data/drugcell_ont.txt',
    header=None
)[[0, 1]]

In [10]:
go.to_csv(
    './graph.csv',
)

In [11]:
genes = set(go[['GO:' not in i for i in list(go[1])]][1])

In [12]:
G = nx.DiGraph()
for i in f.index:
    G.add_node(i, weight=f['HCC44_LUNG'][i])

G.add_nodes_from(genes)
G.add_edges_from(list(go.itertuples(index=False, name=None)))

In [13]:
%%time
res = pd.DataFrame()
end = set(list(go[['GO:' not in i for i in list(go[1])]][0]))

for l in end:
    paths = nx.all_simple_paths(G, source='GO:0008150', target=l)
    path_l = list(paths)
    
    if len(path_l) != 1:
        path = set()
        for i in paths:
            path = path | set(i)
    else:
        path = path_l[0]

    res = pd.concat([
        res,
        pd.DataFrame([
            sum([f['HCC44_LUNG'][i] for i in path]),
            sum([f['OACP4C_OESOPHAGUS'][i] for i in path])
        ], columns=[l], index=['HCC44_LUNG', 'OACP4C_OESOPHAGUS']).T
    ])

CPU times: user 4min 37s, sys: 782 ms, total: 4min 38s
Wall time: 4min 44s


In [14]:
res

,HCC44_LUNG,OACP4C_OESOPHAGUS
GO:0070884,0.16717,1.073820
GO:0009880,0.00000,0.000000
GO:0034033,0.00000,0.000000
GO:0048333,0.00000,0.000000
GO:0050900,0.41270,2.254544
...,...,...
GO:0072080,0.17888,0.829932
GO:0071157,0.00000,0.000000
GO:0008542,0.00000,0.000000
GO:0010035,-0.29153,0.712820


In [15]:
weights = pd.DataFrame()
for i in genes:
    weights = pd.concat([
        weights,
        pd.DataFrame([
            sum([res['HCC44_LUNG'][l] for l in list(go[go[1] == i][0])]),
            sum([res['OACP4C_OESOPHAGUS'][l] for l in list(go[go[1] == i][0])])
        ], index=['HCC44_LUNG', 'OACP4C_OESOPHAGUS'], columns=[i]).T
    ])

In [16]:
weights.to_csv('gene_weight.csv')

In [17]:
pd.concat([
    f,
    weights
]).to_csv('weight.csv')

In [18]:
weights.sort_values('HCC44_LUNG')

,HCC44_LUNG,OACP4C_OESOPHAGUS
TGFB1,-45.436190,56.449968
CAV1,-41.240479,20.584189
RHOA,-36.083979,34.785554
CD36,-35.313462,18.549597
AKT1,-34.474139,37.662417
...,...,...
CDC42BPB,2.062190,3.364768
CDC42BPA,2.062190,3.364768
IL17RA,2.080240,2.680528
FKBP5,2.135700,4.407130


In [19]:
weights.sort_values('OACP4C_OESOPHAGUS')

,HCC44_LUNG,OACP4C_OESOPHAGUS
GRIN3A,-6.824079,-3.375589
CACNA1B,-7.467183,-2.855750
PREPL,-2.723790,-2.777590
ATP2A2,-24.591396,-2.745307
FEN1,-2.524709,-2.475065
...,...,...
NRP1,-7.868931,37.840329
SHH,-18.734794,40.985980
BMP4,-28.414803,47.801763
BCL2,-28.215611,48.632468
